# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

In [15]:
from pathlib import Path
from db_connections import client,db_neo4j,db_redis
from src import mongo,neo4j,utils,redis
from pymongo.errors import ConnectionFailure
import os
import pandas as pd

## Conexión con bases de datos

In [16]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


## LIMPIEZA PREVIA

In [17]:
# --- LIMPIAR DATOS EXISTENTES EN TODAS LAS BASES ---

print("\n1. Limpiando datos existentes...")

# --- MONGO ---
nombre_base = "clase"
client.drop_database(nombre_base)
print("   ✓ Base de datos Mongo eliminada")

# --- REDIS ---
db_redis.flushdb()
print("   ✓ Base de datos Redis limpiada")

# --- NEO4J ---
with db_neo4j.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
print("   ✓ Nodos y relaciones de Neo4j eliminados")

print("\n Todos los datos fueron limpiados correctamente.")



1. Limpiando datos existentes...
   ✓ Base de datos Mongo eliminada
   ✓ Base de datos Redis limpiada
   ✓ Nodos y relaciones de Neo4j eliminados

 Todos los datos fueron limpiados correctamente.


## Carga de Datos

In [18]:
#-------------------------
# Archivos y colecciones
#-------------------------
nombre_base = "clase"
nombre_colecciones = ["usuarios", "destinos", "hoteles", "reservas", "actividades"]
nombre_archivos = ["usuarios.csv", "destinos.csv", "hoteles.csv", "reservas.csv", "actividades.csv"]

for i in range(5):
    ruta = Path("fuentes") / nombre_archivos[i]
    df = utils.lectura_csv(ruta)
    
    if df is not None:
        # Convierte el DataFrame en lista de diccionarios para inserción
        filas = df.to_dict(orient="records")
        
        # -------------------------
        # Inserta datos en MongoDB (Usuarios,Destinos,Hoteles,Reservas,Actividades)
        # -------------------------
        
        # Creo colecccion 
        mongo.crear_coleccion(nombre_base, nombre_colecciones[i], recrear=True)

        # Inserto los datos
        mongo.insertar_muchos_coleccion(nombre_base, nombre_colecciones[i], filas)

        # -------------------------
        # Registro de usuarios conectados en Redis
        # -------------------------

        if nombre_colecciones[i] == "usuarios":
            conectados = redis.guardar_usuarios_conectados(df, 15)
            print(f"✅ Se registran {conectados} usuarios conectados")
        
        # -------------------------
        # Creación de nodos en Neo4j 
        # Solo para usuarios y destinos
        # -------------------------

        # Solo crea en Neo4j nodos Usuarios y Destinos
        if nombre_colecciones[i] in ["usuarios", "destinos"]:
            nombre_nodo = "Usuario" if nombre_colecciones[i] == "usuarios" else "Destino"
            campo_clave = "usuario_id" if nombre_nodo == "Usuario" else "destino_id"

            with db_neo4j.session() as session:
                filas = df.to_dict(orient="records") # Transformo df en dict
                for fila in filas: # Recorro  y agrego
                    nodo = session.execute_write(neo4j.crear_nodo, nombre_nodo, campo_clave, fila)
                print(f"✅ Nodos de {nombre_nodo} creados exitosamente")
        
        # -------------------------
        # Manejo especial de reservas
        # -------------------------

        if nombre_colecciones[i] == "reservas":
            from datetime import datetime

            HOY = datetime.today().date()  # fecha de hoy

            # Crear relaciones VISITO entre Usuario y Destino
            with db_neo4j.session() as session: 
                relaciones_creadas = 0
                for fila in df.to_dict(orient="records"):
                    estado = str(fila.get("estado", "")).strip()
                    fecha = fila.get("fecha_reserva")

                    # Solo si el estado es Confirmada o Pagada y fecha <= hoy
                    if estado in ["Confirmada", "Pagada"] and fecha <= str(HOY):
                        session.execute_write(
                            neo4j.crear_relacion_unidireccional,
                            "Usuario", "usuario_id", fila["usuario_id"],
                            "Destino", "destino_id", fila["destino_id"],
                            "VISITO")                   
            print("✅ Relaciones VISITO creadas exitosamente")

            # Carga en Redis de reservas temporales (estado: Pendiente)
            df_reservas_temporales = df[df["estado"] == ""]
            resultado = redis.carga_masiva_reservas_temporales(df_reservas_temporales)
            print(f"✅ Se insertaron {resultado} reservas temporales")

#Creo relaciones entre usuarios 
ruta_relaciones = Path("fuentes") / "usuarios_relaciones.csv"

usuarios_relaciones_df = utils.lectura_csv(ruta_relaciones)

# Crear relaciones  entre Usuarios
with db_neo4j.session() as session: 
    filas = usuarios_relaciones_df.to_dict(orient="records")
    for fila in filas:
        session.execute_write(
            neo4j.crear_relacion_bidireccional,
            "Usuario", "usuario_id", fila["usuario1"],
            "Usuario", "usuario_id", fila["usuario2"],
            fila["tipo"]
        )
    print("✅ Relaciones entre usuarios creadas exitosamente")


✅ Se insertaron 500 documentos en 'usuarios'.
✅ Se registran 15 usuarios conectados
✅ Nodos de Usuario creados exitosamente
✅ Se insertaron 20 documentos en 'destinos'.
✅ Nodos de Destino creados exitosamente
✅ Se insertaron 100 documentos en 'hoteles'.
✅ Se insertaron 232 documentos en 'reservas'.
✅ Relaciones VISITO creadas exitosamente
✅ Se insertaron None reservas temporales
✅ Se insertaron 120 documentos en 'actividades'.
✅ Relaciones entre usuarios creadas exitosamente


##### Contar la cantidad de documentos existentes en cada colección

In [19]:
for i in range (5):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

La cantidad de documentos de la coleccion usuarios es 500
La cantidad de documentos de la coleccion destinos es 20
La cantidad de documentos de la coleccion hoteles es 100
La cantidad de documentos de la coleccion reservas es 232
La cantidad de documentos de la coleccion actividades es 120


##### Listar datos de las colecciones

In [20]:
# Lista el primer dato de cada colección como prueba de la carga

for i in range (5):
    cursor=mongo.obtener_cursor(nombre_base,nombre_colecciones[i], 1,proyeccion={"_id":0})
    print(f"Colección {nombre_colecciones[i]}:")
    
    mongo.imprimir_cursor(cursor)


Colección usuarios:
{'apellido': 'Cantón',
 'email': 'llopisjose-ignacio@example.org',
 'nombre': 'Juan',
 'telefono': '+34843181960',
 'usuario_id': 1}
Colección destinos:
{'ciudad': 'Córdoba',
 'destino_id': 1,
 'pais': 'Argentina',
 'precio_promedio': 114196,
 'tipo': 'Montaña'}
Colección hoteles:
{'calificacion': 2,
 'ciudad': 'Córdoba',
 'hotel_id': 1,
 'nombre': 'Banca Privada Madrid y asociados S.A. Hotel',
 'precio': 57350,
 'servicios': 'restaurant;pileta;spa;gimnasio'}
Colección reservas:
{'destino_id': 11,
 'estado': 'Cancelada',
 'fecha_reserva': '2024-12-06',
 'precio_total': 82722,
 'reserva_id': 1,
 'usuario_id': 489}
Colección actividades:
{'actividad_id': 1,
 'ciudad': 'Mendoza',
 'nombre': 'Synergized needs-based capacity',
 'precio': 53857,
 'tipo': 'aventura'}


In [21]:
import pandas as pd

# Consulta Cypher para traer relaciones entre usuarios con nombre y apellido
query = """
MATCH (u1:Usuario)-[r:AMIGO_DE]->(u2:Usuario)
RETURN 
    u1.usuario_id AS Usuario1_ID,
    u1.nombre AS Usuario1_Nombre,
    u1.apellido AS Usuario1_Apellido,
    u2.usuario_id AS Usuario2_ID,
    u2.nombre AS Usuario2_Nombre,
    u2.apellido AS Usuario2_Apellido,
    type(r) AS Tipo
ORDER BY u1.usuario_id
"""

# Ejecutar la consulta en Neo4j
with db_neo4j.session() as session:
    resultados = session.run(query)
    relaciones = [dict(r) for r in resultados]

# Convertir a DataFrame
df_relaciones = pd.DataFrame(relaciones)

df_relaciones

,Usuario1_ID,Usuario1_Nombre,Usuario1_Apellido,Usuario2_ID,Usuario2_Nombre,Usuario2_Apellido,Tipo
0,1,Juan,Cantón,2,Amor,Bueno,AMIGO_DE
1,1,Juan,Cantón,3,Leopoldo,Agudo,AMIGO_DE
2,1,Juan,Cantón,4,Cleto,Palomar,AMIGO_DE
3,1,Juan,Cantón,5,Paula,Pardo,AMIGO_DE
4,1,Juan,Cantón,6,Wilfredo,Patiño,AMIGO_DE
...,...,...,...,...,...,...,...
317,495,Gema,Rueda,83,Inés,Cruz,AMIGO_DE
318,498,Rosa María,Losa,401,Kike,Salgado,AMIGO_DE
319,499,Emperatriz,Baena,174,Albina,Morales,AMIGO_DE
320,500,Luis Miguel,Diez,205,Jesús,Benitez,AMIGO_DE
